# Import necessary packages

In [116]:
%matplotlib inline
import sys
import matplotlib.pyplot as plt
import json
import requests
import time
import re
from pathlib import Path
import pandas as pd, seaborn as sns, sklearn, numpy as np
from web3 import Web3
sns.set_style("ticks", {"xtick.major.size": 11, "ytick.major.size": 11})
params = {'font.family': 'serif',
          'pgf.texsystem': 'xelatex',
          'text.usetex': True,
          'text.latex.preamble': "\n".join([r'\usepackage{siunitx}',
                                  r'\usepackage{amsfonts,amsmath,amssymb,amsthm}',
                                  (r'\sisetup{round-mode = places, round-precision = 2,'
                                   +r'zero-decimal-to-integer}'),
                                  r'\usepackage{bm}',
                                  r'\usepackage[utf8]{inputenc}',
                                  ]),
          'legend.fontsize': 11,
          'xtick.labelsize': 11, 'ytick.labelsize': 11, 'axes.labelsize': 11}
plt.rcParams.update(params)


ABI_ENDPOINT = "https://api.bscscan.com/api?module=contract&action=getabi&address="
bsc = "https://bsc-dataseed.binance.org/"
w3 = Web3(Web3.HTTPProvider(bsc))
print("Connected to web3:", w3.isConnected())
token_regex = re.compile('"name":"(?:name|symbol)"')


Connected to web3: True


# Define useful methods

In [111]:
def get_token_name_symbol(address):
    # TODO: use bscscan API - https://github.com/pcko1/bscscan-python
    status_success = False
    sleep_time = 5
    while not status_success:
        response = requests.get(f"{ABI_ENDPOINT}{address}")
        response_json = response.json()
        status_success = bool(int(response_json['status']))
        response_result = response_json['result']
        time.sleep(sleep_time)
        if not status_success:
            print(f"Request failed with response: {response_json}")
            if "max rate limit reached" in response_result.lower():
                sleep_time += 1
                print(f"Increasing sleep time to {sleep_time + 1}")
            if "contract source code not verified" in response_result.lower():
                print("Contract not valid")
                return None, None, False
    
    try:
        abi_json = json.loads(response_result)
    except Exception as e:
        print(response_json)
        sys.exit()
    contract = w3.eth.contract(address, abi=abi_json)
    # ensure we do not hit rate limit

    regex_matches = token_regex.findall(response_result)
    checks = [any([check_value == regex_match for regex_match in regex_matches])
              for check_value in ['"name":"name"', '"name":"symbol"']]
    name = contract.functions.name().call() if checks[0] else None
    symbol = contract.functions.symbol().call() if checks[1] else None
    return name, symbol, True

def address_to_symbol(address):
    checksum_address = web3.toChecksumAddress(address)
    token_constract = w3.eth.contract(checksum_address)

# Load and manipulate data

In [107]:
file_name = Path('sample_prices_with_symbols.csv')
if not file_name.exists():
    file_name = 'sample_prices.csv'
    df = pd.read_csv(file_name)
    df = df.drop('Unnamed: 0', axis=1)
    df['mean'] = df[['high','low']].mean(axis=1)
    address_to_symbol_name = {address: get_token_name_symbol(address) for address in df['token'].unique()}
    # list comprehension is preferred - https://stackoverflow.com/questions/54432583/when-should-i-not-want-to-use-pandas-apply-in-my-code
    df[['name','symbol', 'verified']] = [address_to_symbol_name[address] for address in df['token']]
    df.to_csv('./sample_prices_with_symbols.csv')
else:
    df = pd.read_csv(file_name)


# Basic Data

In [114]:
token_df = df[['token', 'liqToken', 'name', 'symbol']]
num_uniques = token_df.nunique()
print(f'Shape (datapoints x columns): {df.shape}')
print(f'Features: {df.columns.values}')
print(f'Number of unique values:\n{num_uniques}')
# TODO replace unicode format ensure format is either raw or unicode

Shape (datapoints x columns): (1167, 10)
Features: ['Unnamed: 0' 'token' 'liqToken' 'timestamp' 'high' 'low' 'mean' 'name'
 'symbol' 'verified']
Number of unique values:
token       665
liqToken      2
name        578
symbol      568
dtype: int64


# How often does each token occur

In [109]:
sns.histplot(token_df, x='symbol')


<AxesSubplot:xlabel='symbol', ylabel='Count'>

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f60f09b1af0> (for post_execute):


RuntimeError: latex was not able to process the following string:
b'REDFEG \\U0001f98d'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(../efac6f9fe94e3c4c099c769470993b1e.tex
LaTeX2e <2020-10-01> patch level 4
L3 programming layer <2021-01-09> xparse <2020-03-03>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2020/04/10 v1.4m Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/share/texmf/tex/latex/cm-super/type1ec.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex

RuntimeError: latex was not able to process the following string:
b'REDFEG \\U0001f98d'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(../efac6f9fe94e3c4c099c769470993b1e.tex
LaTeX2e <2020-10-01> patch level 4
L3 programming layer <2021-01-09> xparse <2020-03-03>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2020/04/10 v1.4m Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/share/texmf/tex/latex/cm-super/type1ec.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex

<Figure size 432x288 with 1 Axes>

In [6]:
sns.lineplot(data=df, x='timestamp',y='mean', hue='token', legend=None)

<AxesSubplot:xlabel='timestamp', ylabel='mean'>

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f3e51947af0> (for post_execute):


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(../df3b86785ccbddb56c0e274ced8af12d.tex
LaTeX2e <2020-10-01> patch level 4
L3 programming layer <2021-01-09> xparse <2020-03-03>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2020/04/10 v1.4m Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/share/texmf/tex/latex/cm-super/type1ec.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifvtex.sty
(/us

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(../df3b86785ccbddb56c0e274ced8af12d.tex
LaTeX2e <2020-10-01> patch level 4
L3 programming layer <2021-01-09> xparse <2020-03-03>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2020/04/10 v1.4m Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/share/texmf/tex/latex/cm-super/type1ec.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifvtex.sty
(/us

<Figure size 432x288 with 1 Axes>